In [400]:
import shutil
import os
import zipfile
import urllib.request
from typing import Optional, Sequence, cast, ClassVar, cast
from dataclasses import dataclass

import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import numpy.typing as npt
import cv2 as cv
import tqdm

import scipy.spatial.distance

import sklearn.metrics
import sklearn.manifold
import sklearn.decomposition

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline

import random

import skimage.feature

from standalone_preprocessing_library import *

plotly.offline.init_notebook_mode(connected=True)

In [401]:
SEED = 2024
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

Various constants

In [2]:
DATA_DIR_PATH = "data"
DATASET_PATH = "dataset.zip"
TRAINING_DATA_CSV_PATH = os.path.join(DATA_DIR_PATH, "data.csv")

MODEL_DIR_PATH = "models"
EXPRESSION_RECOGNITION_MODEL_PATH = os.path.join(MODEL_DIR_PATH, "expression_recognition.keras")
LBFMODEL_PATH = os.path.join(MODEL_DIR_PATH, "lbfmodel.yaml")
CASCADE_CLASSIFIER_PATH = os.path.join(MODEL_DIR_PATH, "haarcascade_frontalface_default.xml")

STANDARD_DIMENSIONS = Dimension(240, 240)
FACE_DIMENSIONS = Dimension(120, 120)
PREVIEW_DIMENSIONS = Dimension(500, 500)

LBP_HISTOGRAM_BIN_COUNT = 8
LBP_GRID_SIZE = (8,8)

In [3]:
class SupportedFacialExpressionLabels:
  Ours = ["angry", "disgusted", "happy", "neutral", "sad", "surprised"]
  Fer2013 = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"] 
  CkPlus = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral", "Contempt"]

CLASS_NAMES = SupportedFacialExpressionLabels.Ours

Utility functions

In [4]:
def get_files_in_folder(basepath: str)->list[str]:
  if not os.path.exists(basepath):
    return []
  file_paths = [os.path.join(basepath, filename) for filename in os.listdir(basepath)]
  return [path for path in file_paths if os.path.isfile(path)]

def partition_grid(self, rows: int, cols: int)->list["Rectangle"]:
  row_delta = int(self.width / rows)
  col_delta = int(self.height / cols)
  rects: list["Rectangle"] = []
  for row in range(rows):
    for col in range(cols):
      row_start = row * row_delta
      col_start = col * col_delta
      rect = Rectangle(col_start, row_start, col_start + col_delta, row_start + row_delta)
      rects.append(rect)
  return rects

@dataclass
class FaceLandmark:
  face_shape: npt.NDArray
  eyes: npt.NDArray
  eyebrows: npt.NDArray
  nose: npt.NDArray
  lips: npt.NDArray
  dims: Dimension

  @property
  def feature_points(self)->npt.NDArray:
    return np.vstack([self.eyes, self.eyebrows, self.nose, self.lips])

  def as_feature_vector(self)->npt.NDArray:
    # https://arxiv.org/pdf/1812.04510
    # 17 points are dedicated for the shape of the face, which we don't really need.
    normalized_points = self.feature_points / np.array((self.dims.width, self.dims.height))
    interdistance_map = scipy.spatial.distance.cdist(normalized_points, normalized_points, "euclidean").flatten()
    # All diagonal values are excluded
    excluded_points = np.eye(len(self.feature_points)).flatten() == 1

    # Square the interdistance map to make larger differences more prominent
    interdistance_map = np.power(interdistance_map[~excluded_points], 2)

    # Also calculate the distance to the average point in the face
    average_point = normalized_points.mean(axis=0)
    distances_to_center = scipy.spatial.distance.cdist(np.array([average_point]), normalized_points, "euclidean")[0]

    feature_vector = np.hstack((interdistance_map, distances_to_center))

    return feature_vector


  EYE_COLOR: ClassVar[tuple[int, int, int]] = (0, 0, 255)
  LIP_COLOR: ClassVar[tuple[int, int, int]] = (0, 255, 0)
  NOSE_COLOR: ClassVar[tuple[int, int, int]] = (255, 0, 0)
  FACE_SHAPE_COLOR: ClassVar[tuple[int, int, int]] = (255, 255, 0)
  EYEBROW_COLOR: ClassVar[tuple[int, int, int]] = (0, 255, 255)

  def project_point(self, point: npt.NDArray, rect: Optional[Rectangle] = None)->Sequence[int]:
    if rect is None:
      return cast(Sequence[int], point.astype(np.int32))
    projected_point = point * rect.dimensions.ndarray / self.dims.ndarray
    return cast(Sequence[int], (projected_point + rect.p0.ndarray).astype(np.int32))
  
  def draw_on(self, img: cv.typing.MatLike, *, offset: Optional[Rectangle] = None):
    for point in self.eyes:
      cv.circle(img, self.project_point(point, offset), 1, self.EYE_COLOR, -1)
    for point in self.lips:
      cv.circle(img, self.project_point(point, offset), 1, self.LIP_COLOR, -1)
    for point in self.nose:
      cv.circle(img, self.project_point(point, offset), 1, self.NOSE_COLOR, -1)
    for point in self.face_shape:
      cv.circle(img, self.project_point(point, offset), 1, self.FACE_SHAPE_COLOR, -1)
    for point in self.eyebrows:
      cv.circle(img, self.project_point(point, offset), 1, self.EYEBROW_COLOR, -1)

  @staticmethod
  def from_raw_landmark(points: npt.NDArray, dims: Dimension):
    return FaceLandmark(
      face_shape=points[:17],
      eyebrows=points[17:27],
      nose=points[27:36],
      eyes=points[36:48],
      lips=points[48:],
      dims=dims
    )
  
def face_alignment(img: cv.typing.MatLike, landmark: FaceLandmark):
  # https://pyimagesearch.com/2017/05/22/face-alignment-with-opencv-and-python/
  dims = Dimension.from_shape(img.shape)
  desired_left_eye = Point(int(dims.width * 0.22), int(dims.height * 0.25))
  desired_right_eye_x = FACE_DIMENSIONS.width - desired_left_eye.x

  left_eye_avg = landmark.eyes[0:6].mean(axis=0)
  right_eye_avg = landmark.eyes[6:].mean(axis=0)

  delta = right_eye_avg - left_eye_avg
  angle = np.degrees(np.arctan2(delta[1], delta[0]))

  dist = np.sqrt(delta[0] ** 2 + delta[1] ** 2)
  desired_dist = desired_right_eye_x - desired_left_eye.x
  scale = desired_dist / dist

  eyes_center = np.array([left_eye_avg, right_eye_avg]).mean(axis=0)
  rotation_matrix = cv.getRotationMatrix2D(eyes_center, angle, scale)

  translation_x = FACE_DIMENSIONS.width * 0.5
  translation_y = desired_left_eye.y
  rotation_matrix[0, 2] += (translation_x - eyes_center[0])
  rotation_matrix[1, 2] += (translation_y - eyes_center[1])

  img = cv.warpAffine(img, rotation_matrix, FACE_DIMENSIONS.tuple, flags=cv.INTER_CUBIC)

  return img


Load various preprocessing models

In [5]:
# https://medium.com/analytics-vidhya/facial-landmarks-and-face-detection-in-python-with-opencv-73979391f30e
LBFMODEL_URL = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"
if not os.path.exists(LBFMODEL_PATH):
  print(f"Cannot find any LBFmodel installation. Installing from {LBFMODEL_URL} to {LBFMODEL_PATH}")
  urllib.request.urlretrieve(LBFMODEL_URL, LBFMODEL_PATH)
landmark_model = cv.face.createFacemarkLBF()
landmark_model.loadModel(LBFMODEL_PATH)

In [6]:
CASCADE_CLASSIFIER_URL = "https://raw.githubusercontent.com/opencv/opencv/refs/heads/4.x/data/haarcascades/haarcascade_frontalface_default.xml"
if not os.path.exists(CASCADE_CLASSIFIER_PATH):
  urllib.request.urlretrieve(CASCADE_CLASSIFIER_URL, CASCADE_CLASSIFIER_PATH) 
locator_model = cv.CascadeClassifier(CASCADE_CLASSIFIER_PATH)
locator_model.load(CASCADE_CLASSIFIER_PATH)

clahe = cv.createCLAHE(tileGridSize=(8, 8), clipLimit=2.0)

Dump the contents of the zip file into the data folder. You don't have to run this if this has been performed.

In [7]:
if (os.path.exists(DATA_DIR_PATH)):
  shutil.rmtree(DATA_DIR_PATH)
with zipfile.ZipFile(DATASET_PATH, 'r') as zip_ref:
  zip_ref.extractall(DATA_DIR_PATH)

List all folder contents

In [8]:
@dataclass
class TrainDataEntry:
  path: str
  label: int

entries: list[TrainDataEntry] = []
for folder in os.scandir(DATA_DIR_PATH):
  if not folder.is_dir():
    continue
  try:
    expression = CLASS_NAMES.index(folder.name)
  except ValueError:
    print(f"Skipping the inclusion of {folder.name} in the dataset.")
    continue

  entries.extend(map(
    lambda fpath: TrainDataEntry(path=fpath, label=expression),
    get_files_in_folder(folder.path)
  ))

Execute data preprocessing steps for our dataset

In [9]:
skipped = 0
list_data: list[npt.NDArray] = []
list_labels: list[int] = []
for entry in tqdm.tqdm(entries, desc="Building dataset from images"):
  original = cv.imread(entry.path)
  
  # Preprocessing
  img_resized = resize_image(original, STANDARD_DIMENSIONS)
  img_grayscale = cv.cvtColor(img_resized, cv.COLOR_BGR2GRAY)
  img_clahe = clahe.apply(img_grayscale)

  # Extract face coordinates
  face_coordinates = locator_model.detectMultiScale(img_clahe)
  face_rects = list(Rectangle.from_tuple(coords) for coords in face_coordinates)
  face_rects.sort(key=lambda x: x.area)
  saved_face_rects: list[Rectangle] = []
  # Prevent overlapping face rectangles
  for face_rect_a in face_rects:
    is_overlapping = False
    for face_rect_b in saved_face_rects:
      IOU = face_rect_a.intersection_with_union(face_rect_b)
      if IOU > 0.4:
        is_overlapping = True
        break

    if not is_overlapping:
      saved_face_rects.append(face_rect_a)

  # Get faces from original image
  faces = list(
    img_clahe[pos.slice]
    for pos in face_rects
  )

  features: list[npt.NDArray] = []
  for raw_face_img in faces:
    # Additional preprocessing
    face_resized = cv.resize(raw_face_img, FACE_DIMENSIONS.tuple, interpolation=cv.INTER_CUBIC)
    face_blurred = cv.filter2D(face_resized, -1, GAUSSIAN_3X3_KERNEL)
    face_preprocessed = cv.filter2D(face_blurred, -1, SHARPEN_KERNEL)

    # Get face landmarks
    _, raw_face_landmarks = landmark_model.fit(face_preprocessed, np.array(((0, 0, face_preprocessed.shape[0], face_preprocessed.shape[1]),)))
    face_landmark_points: npt.NDArray = raw_face_landmarks[0][0]
    face_landmark = FaceLandmark(
      face_shape=face_landmark_points[:17],
      eyebrows=face_landmark_points[17:27],
      nose=face_landmark_points[27:36],
      eyes=face_landmark_points[36:48],
      lips=face_landmark_points[48:],
      dims=Dimension.from_shape(face_preprocessed.shape)
    )
    face_dims = Dimension.from_shape(raw_face_img.shape)
    face_landmark = FaceLandmark.from_raw_landmark(face_landmark_points, face_dims)
    face_aligned = face_alignment(face_preprocessed, face_landmark)

    # Split image to grids for LBP
    lbp_grid_rects = partition_grid(face_dims, *LBP_GRID_SIZE)
    histograms: list[npt.NDArray] = []

    # Perform LBP
    lbp_image: npt.NDArray = skimage.feature.local_binary_pattern(face_aligned, 8, 1)
    for lbp_grid_rect in lbp_grid_rects:
      chunk = lbp_image[lbp_grid_rect.slice]

      if chunk.size == 0:
        histograms.append(np.full((LBP_HISTOGRAM_BIN_COUNT,), 0))
        continue
      histograms.append(scipy.ndimage.histogram(chunk, 0, 255, LBP_HISTOGRAM_BIN_COUNT) / chunk.size)

    feature_vector = np.hstack(histograms)
    features.append(feature_vector)

  if len(features) == 0:
    print(f"\nSkipping {entry.path} because no faces were found in the image.")
    skipped += 1
    continue
  list_data.extend(features)
  list_labels.extend([entry.label] * len(features))

print(f"Skipped over {skipped} images because no faces were found in the images.")

data = np.array(list_data)
labels = np.array(list_labels).reshape((-1, 1))
dfdata = np.hstack((labels, data))
df = pd.DataFrame(dfdata, columns=[
  "label",
  *map(lambda idx: f'feature-{idx + 1}', range(data.shape[1])),
])

df.to_csv(TRAINING_DATA_CSV_PATH, index=False)

Building dataset from images:   0%|          | 1/1089 [00:00<15:11,  1.19it/s]


Skipping data\angry\20240927_173906.jpg because no faces were found in the image.


Building dataset from images:   1%|          | 8/1089 [00:01<02:08,  8.42it/s]


Skipping data\angry\20241008_180500.jpg because no faces were found in the image.

Skipping data\angry\20241008_180548.jpg because no faces were found in the image.


Building dataset from images:   1%|▏         | 16/1089 [00:02<01:09, 15.33it/s]

...


Skipping data\surprised\terkejut_4.jpg because no faces were found in the image.


Building dataset from images:  97%|█████████▋| 1055/1089 [01:27<00:07,  4.28it/s]


Skipping data\surprised\terkejut_5.jpg because no faces were found in the image.


Building dataset from images: 100%|██████████| 1089/1089 [01:33<00:00, 11.63it/s]


Skipped over 75 images because no faces were found in the images.


Start training process

In [404]:
df = pd.read_csv(TRAINING_DATA_CSV_PATH, index_col=False)
labels = df["label"]
df = df.drop(columns=["label"])
df

,feature-1,feature-2,feature-3,feature-4,feature-5,feature-6,feature-7,feature-8,feature-9,feature-10,...,feature-503,feature-504,feature-505,feature-506,feature-507,feature-508,feature-509,feature-510,feature-511,feature-512
0,0.376731,0.041551,0.016620,0.058172,0.030471,0.000000,0.072022,0.404432,0.479224,0.008310,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.408163,0.090703,0.018141,0.083900,0.006803,0.002268,0.081633,0.308390,0.512472,0.036281,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.503086,0.089506,0.018519,0.080247,0.015432,0.000000,0.061728,0.231481,0.444444,0.046296,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.404959,0.047521,0.020661,0.068182,0.045455,0.004132,0.076446,0.332645,0.438017,0.066116,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.371901,0.057851,0.020661,0.082645,0.045455,0.004132,0.053719,0.363636,0.458678,0.039256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,0.173469,0.076531,0.040816,0.193878,0.127551,0.010204,0.178571,0.198980,0.153061,0.122449,...,0.229592,0.275510,0.107143,0.107143,0.025510,0.056122,0.107143,0.020408,0.428571,0.147959
1078,0.146667,0.111111,0.035556,0.168889,0.120000,0.013333,0.186667,0.217778,0.191111,0.128889,...,0.546667,0.128889,0.342222,0.093333,0.057778,0.142222,0.013333,0.008889,0.057778,0.284444
1079,0.142857,0.117347,0.040816,0.178571,0.117347,0.015306,0.193878,0.193878,0.188776,0.081633,...,0.250000,0.244898,0.163265,0.066327,0.020408,0.061224,0.091837,0.010204,0.418367,0.168367
1080,0.158163,0.096939,0.040816,0.173469,0.102041,0.010204,0.198980,0.219388,0.168367,0.086735,...,0.239796,0.244898,0.158163,0.076531,0.030612,0.056122,0.081633,0.010204,0.454082,0.132653


Data Visualization

In [405]:
tsne = sklearn.manifold.TSNE(2)
data_points2d = tsne.fit_transform(df)

labels_series = pd.Series(labels, name="Label").replace(range(len(CLASS_NAMES)), CLASS_NAMES) # type: ignore

data_points2d_df = pd.DataFrame(data_points2d, columns=["x", "y"])
visdf = pd.concat([data_points2d_df, labels_series], axis=1)

fig = px.scatter(visdf, x="x", y="y", color="Label")
fig.show()

In [406]:
dataset = tf.data.Dataset.from_tensor_slices((df, labels))
train_dataset, test_dataset = keras.utils.split_dataset(dataset, shuffle=True, seed=42, left_size=0.7)
train_dataset, validation_dataset = keras.utils.split_dataset(train_dataset, shuffle=True, seed=42, left_size=0.7)

train_dataset: tf.data.Dataset = train_dataset.shuffle(100).batch(16)
test_dataset: tf.data.Dataset = test_dataset.batch(16)
validation_dataset: tf.data.Dataset = validation_dataset.batch(16)

dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

In [461]:
model = keras.Sequential([
  keras.layers.Input(shape=dataset.element_spec[0].shape),
  keras.layers.Dense(256, activation="relu"),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(128, activation="relu"),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(64, activation="relu"),
  keras.layers.Dense(6, activation="softmax"),
])

model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.001), # type: ignore
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=["accuracy"],
)

model.summary()

Model: "sequential_44"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_162 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_88 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_163 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_89 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_164 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_165 (Dense)               │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 172,870 (675.27 KB)

 Trainable params: 172,870 (675.27 KB)

 Non-trainable params: 0 (0.00 B)

In [462]:
history = model.fit(
  train_dataset, epochs=100,
  validation_data=validation_dataset,
)

Epoch 1/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1999 - loss: 1.8062 - val_accuracy: 0.2423 - val_loss: 1.7575
Epoch 2/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2284 - loss: 1.7688 - val_accuracy: 0.2070 - val_loss: 1.7490
Epoch 3/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2604 - loss: 1.7431 - val_accuracy: 0.2731 - val_loss: 1.7253
Epoch 4/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2827 - loss: 1.7245 - val_accuracy: 0.2555 - val_loss: 1.7041
Epoch 5/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2974 - loss: 1.6905 - val_accuracy: 0.2731 - val_loss: 1.6853
Epoch 6/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2824 - loss: 1.7109 - val_accuracy: 0.2952 - val_loss: 1.6749
Epoch 7/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3375 - loss: 1.6446 - val_accuracy: 0.3656 - val_loss: 1.6131
Epoch 8/60
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3604 - loss: 1.5560 - val_accuracy: 0.4053 - val_loss

In [477]:
history_df = pd.DataFrame(history.history)

fig = px.line(history_df, x=history_df.index, y="accuracy", markers=True, title="Training Accuracy")
fig2 = px.line(history_df, x=history_df.index, y="val_accuracy", markers=True)

fig.update_traces(dict(
  marker=dict(
    color="blue"
  ),
  line=dict(
    color="blue",
  ),
))
fig2.update_traces(dict(
  marker=dict(
    color="orange"
  ),
  line=dict(
    color="orange",
  ),
))
fig.add_traces(fig2.data)
fig.data[0]["name"] = "Accuracy" # type: ignore
fig.data[1]["name"] = "Validation Accuracy" # type: ignore
fig.update_yaxes(dict(range=[0,1.2], title="Accuracy"))
fig.update_xaxes(dict(title="Epochs"))

lines = [
  dict(y=history_df["val_accuracy"].median(), is_validation=True, label=f"Median Validation Accuracy"),
  dict(y=history_df["accuracy"].median(), is_validation=False, label=f"Median Accuracy"),
  dict(y=history_df["val_accuracy"].max(), is_validation=True, label=f"Max Validation Accuracy"),
  dict(y=history_df["accuracy"].max(), is_validation=False, label=f"Max Accuracy"),
]
VALIDATION_COLOR = "rgba(255, 165, 0,"
TRAINING_COLOR = "rgba(0, 0, 255,"
for line in lines:
  base_color = VALIDATION_COLOR if line['is_validation'] else TRAINING_COLOR
  fig.add_hline(
    y=line["y"],
    line_color=f"{base_color} 0.5)",
    line_dash="dash",
    label=dict(text=f"{line['label']} ({line['y']:.2f})", textposition="start", font=dict(color=f"{base_color} 1)")),
  )

fig.update_traces(dict(
  showlegend=True
))

fig.show()

In [464]:
history_df = pd.DataFrame(history.history)

fig = px.line(history_df, x=history_df.index, y="loss", markers=True, title="Training Loss")
fig2 = px.line(history_df, x=history_df.index, y="val_loss", markers=True)

fig.update_traces(dict(
  marker=dict(
    color="blue"
  ),
  line=dict(
    color="blue",
  ),
))
fig2.update_traces(dict(
  marker=dict(
    color="orange"
  ),
  line=dict(
    color="orange",
  ),
))
fig.add_traces(fig2.data)
fig.update_xaxes(dict(title="Epochs"))
fig.update_yaxes(dict(title="Loss"))
fig.data[0]["name"] = "Loss" # type: ignore
fig.data[1]["name"] = "Validation Loss" # type: ignore


lines = [
  dict(y=history_df["val_loss"].median(), is_validation=True, label=f"Median Validation Loss"),
  dict(y=history_df["loss"].median(), is_validation=False, label=f"Median Loss"),
  dict(y=history_df["val_loss"].min(), is_validation=True, label=f"Min Validation Loss"),
  dict(y=history_df["loss"].min(), is_validation=False, label=f"Min Loss"),
]
VALIDATION_COLOR = "rgba(255, 165, 0,"
TRAINING_COLOR = "rgba(0, 0, 255,"
for line in lines:
  base_color = VALIDATION_COLOR if line['is_validation'] else TRAINING_COLOR
  fig.add_hline(
    y=line["y"],
    line_color=f"{base_color} 0.5)",
    line_dash="dash",
    label=dict(text=f"{line['label']} ({line['y']:.2f})", textposition="start", font=dict(color=f"{base_color} 1)")),
  )

fig.update_traces(dict(
  showlegend=True
))

fig.show()

In [473]:
from typing import cast

confidences = model.predict(test_dataset)
predictions = np.argmax(confidences, axis=1)

labels = cast(list[int], list(test_dataset.unbatch().map(lambda x, y: y).as_numpy_iterator()))

cmat = sklearn.metrics.confusion_matrix(labels, predictions, normalize="true")

fig = px.imshow(cmat, x=CLASS_NAMES, y=CLASS_NAMES, zmin=0, zmax=1)
stringify = np.vectorize(lambda x: f"{x * 100:.2f}%")
fig.update_traces(dict(
  text=stringify(cmat),
  texttemplate="%{text}"
))
fig.show()

sklearn.metrics.classification_report(labels, predictions, output_dict=True, target_names=CLASS_NAMES)

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


{'angry': {'precision': 0.725,
  'recall': 0.6304347826086957,
  'f1-score': 0.6744186046511628,
  'support': 46.0},
 'disgusted': {'precision': 0.8367346938775511,
  'recall': 0.7592592592592593,
  'f1-score': 0.7961165048543689,
  'support': 54.0},
 'happy': {'precision': 0.7887323943661971,
  'recall': 0.8235294117647058,
  'f1-score': 0.8057553956834532,
  'support': 68.0},
 'neutral': {'precision': 0.7333333333333333,
  'recall': 0.7021276595744681,
  'f1-score': 0.717391304347826,
  'support': 47.0},
 'sad': {'precision': 0.6774193548387096,
  'recall': 0.7777777777777778,
  'f1-score': 0.7241379310344828,
  'support': 54.0},
 'surprised': {'precision': 0.7241379310344828,
  'recall': 0.75,
  'f1-score': 0.7368421052631579,
  'support': 56.0},
 'accuracy': 0.7476923076923077,
 'macro avg': {'precision': 0.747559617908379,
  'recall': 0.7405214818308177,
  'f1-score': 0.7424436409724086,
  'support': 325.0},
 'weighted avg': {'precision': 0.7500508069297761,
  'recall': 0.74769230

In [486]:
predictions_series = pd.Series(predictions, name="Predictions").replace(range(len(CLASS_NAMES)), CLASS_NAMES) #type: ignore
labels_series = pd.Series(labels, name="Label").replace(range(len(CLASS_NAMES)), CLASS_NAMES) # type: ignore
discrete_color_map = dict(zip(CLASS_NAMES, px.colors.qualitative.Plotly))
symbol_map = {1: "circle", 0: "x"}

correct_state = pd.Series(labels_series == predictions_series, name="Correct")
visdf = pd.concat([data_points2d_df, labels_series, predictions_series, correct_state], axis=1)
visdf2 = pd.concat([data_points2d_df, labels_series, predictions_series, correct_state], axis=1)
fig = px.scatter(visdf, x="x", y="y", color="Predictions", hover_data=["Label"], symbol="Correct", color_discrete_map=discrete_color_map, symbol_map=symbol_map)
fig2 = px.scatter(visdf, x="x", y="y", color="Label", opacity=0.2, color_discrete_map=discrete_color_map)

fig2.update_traces(dict(
  marker=dict(
    size=12,
  ),
  hoverinfo="skip",
  showlegend=False
))

fig2.add_traces(fig.data)

fig2.show()

In [487]:
tsne_confidences = sklearn.manifold.TSNE()
confidences_points = tsne_confidences.fit_transform(confidences)

confidences_points_df = pd.DataFrame(confidences_points, columns=["x", "y"])
percentagify = np.vectorize(lambda x: f"{x*100:.2f}%")
confidences_df = pd.DataFrame(percentagify(confidences), columns=CLASS_NAMES)
confidences_df["Max Confidence"] = confidences.max(axis=1)
visdf = pd.concat([confidences_points_df, confidences_df, predictions_series, labels_series, correct_state], axis=1)



fig = px.scatter(visdf, x="x", y="y", color="Predictions", symbol="Correct", symbol_map=symbol_map, color_discrete_map=discrete_color_map, hover_data=[*CLASS_NAMES, "Label"])
fig2 = px.scatter(visdf, x="x", y="y", color="Label", opacity=0.2, color_discrete_map=discrete_color_map)
fig3 = px.density_contour(visdf, x="x", y="y")
fig2.update_traces(dict(
  marker=dict(
    size=12,
  ),
  hoverinfo="skip",
  hovertemplate=None,
  showlegend=False
))
fig3.update_traces(dict(
  hoverinfo="skip",
  hovertemplate=None
))

fig3.add_traces(fig2.data)
fig3.add_traces(fig.data)
fig3.show()

In [488]:
model.save(EXPRESSION_RECOGNITION_MODEL_PATH)